# TP2
### Problema 2

O Conway’s Game of Life é um exemplo bastante conhecido de um autómato celular. Neste problema vamos modificar as regras do autómato da seguinte forma:
- O espaço de estados é finito definido por uma grelha de células booleanas (morta=0/viva=1) de dimensão $\,N\times N\,$ (com $N>3$) identificadas por índices $\,(i,j)\in \{1..N\}$.  Estas $\;N^2\;$ células são aqui referidas como “normais”;
<br><br>
- Adicionalmente existem $\,2N+1\,$ “células da borda” que correspondem a um dos índices, $i$ ou $j$, ser zero. As células da borda têm valores constantes que, no estado inicial, são gerados aleatoriamente com uma probabilidade $\,\rho\,$ de estarem vivas;
<br><br>
- As células normais o autómato modificam o estado de acordo com a regra “B3/S23”: i.e. a célula nasce (passa de $0$ a $1$) se tem exatamente 3 vizinhos vivos e sobrevive (mantém-se viva) se o número de vizinhos vivos é 2 ou 3, caso contrário morre ou continua morta;

In [13]:
from pysmt.shortcuts import *
from pysmt.typing import BOOL, INT

### Inputs
São parâmetros do problema os parâmetros N, $\rho$ e a posição do centro.

In [14]:
N = 10 #dimensão da grelha (NxN) (N > 3)
#ro = 0.5 #probabilidade ró de estarem vivas.
centro = (5,5) #posição do centro

        

In [15]:
quadrado = [(centro[0]+1, centro[1]), (centro[0], centro[1]+1),
           (centro[0]-1, centro[1]), (centro[0], centro[1]-1),
           (centro[0]+1, centro[1]+1), (centro[0]-1, centro[1]-1),
           (centro[0]+1, centro[1]-1), (centro[0]-1, centro[1]+1),
           (centro[0], centro[1])]
'''A = {}
for i in range(1,N+1):
    for j in range(1,N+1):
        if (i,j) in quadrado:
            A[i,j] = Symbol(str(i)+str(j), BOOL)'''

'A = {}\nfor i in range(1,N+1):\n    for j in range(1,N+1):\n        if (i,j) in quadrado:\n            A[i,j] = Symbol(str(i)+str(j), BOOL)'

A função `declaracao` cria a i-ésima cópia das variáveis de estado, guardando-as num dicionário.

In [16]:
def declaracao(i, N):
    state = {}
    state['nodo'] = Symbol('nodo'+str(i), INT)
    state['grelha'] = {}
    for i in range(N+1):
        for j in range(N+1):
            state['grelha'][i,j] = Symbol('grelha_'+str(i)+'_'+str(j), BOOL)
            
    return state

A função `inicializacao`, testa se um dado estado é um possível estado inicial do programa.

In [29]:
def inicializacao(state, N, quadrado):
    expr = [Equals(state['nodo'], Int(0))]
    
    for i in range(1, N+1):
        for j in range(1, N+1):
            if (i,j) not in quadrado:
                expr.append(Iff(state['grelha'][i,j], Bool(False)))
            else:
                expr.append(Iff(state['grelha'][i,j], Bool(True)))
    
    return And(elem for elem in expr)

In [40]:
def transforma_grelha(grelha_curr, grelha_prox, N):
    sucessos = []
    excecoes = [0,N+1]
    
    for i in range(1, N+1):
        for j in range(1, N+1):
            
            if Iff(grelha_curr[i,j], Bool(False)):
                vizinhos = [grelha_curr[i+x,j+y] for x in range(-1,2) for y in range(-1,2)]
                #REMOVER EXCECOES
                if vizinhos.count(Bool(True)) == 3:
                    sucessos.append(Iff(grelha_prox[i,j], Bool(True)))
                else:
                    sucessos.append(Iff(grelha_prox[i,j], Bool(False)))
                    
            elif Iff(grelha_curr[i,j], Bool(True)):
                vizinhos = [grelha_curr[i+x,j+y] for x in range(-1,2) for y in range(-1,2)]
                
                if vizinhos.count(Bool(True)) >= 2:
                    sucessos.append(Iff(grelha_prox[i,j], Bool(True)))
                else:
                    sucessos.append(Iff(grelha_prox[i,j], Bool(False)))
                    
    return And(sucessos[i] for i in range(len(sucessos)))

In [31]:
def transicao(curr, prox, N):
    
    c1 = And(Equals(curr['nodo']+1, prox['nodo']),
             transforma_grelha(curr['grelha'], prox['grelha'], N))
    
    return c1

In [39]:
def gera_traco(declaracao, inicializacao, transicao, k, N, quadrado):

    with Solver(name = "z3") as solver:
        # definir o traço de estados(arestas)
        traco = [declaracao(i, N) for i in range(k)]
        
        # adicionar o estado inicial
        solver.add_assertion(inicializacao(traco[0], N, quadrado))
        
        # adicionar a função de transição
        for i in range(k-1):
            solver.add_assertion(transicao(traco[i], traco[i+1], N)) # condição ; # tem de ser verdadeira consoante o nosso estado atual e o seguinte
        
        """
        if s.solve():
            m = s.get_model()
            for n, v in m:
                print(f'{n} = {v}')
        """
        
        for i in range(k):
            print("Passo: ", i)
            for v in trace[i]:
                print(v, "=", solver.get_value(trace[i][v]))
            print("-------------")
        
        
        pass
    # completar
    
  
                
gera_traco(declaracao,inicializacao,transicao,10,N,quadrado)

NameError: name 'x' is not defined